In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from time import time
from tqdm.auto import tqdm

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

e:\Poli NUOVO\Computational Intelligence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [3]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy Algorithm (Multiple tries) - Faster but not optimal solution

This algorithm provides a fast solution using a greedy approach, which we try to improve by running it multiple times and randomizing the start at each iteration, and ultimately choosing the best.

In [4]:
def greedy_tsp(start_city):
    unvisited = set(range(len(CITIES)))
    unvisited.remove(start_city)
    tour = [start_city]
    current_city = start_city
    
    while unvisited:
        next_city = min(unvisited, key=lambda city: DIST_MATRIX[current_city, city])
        tour.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city

    tour.append(start_city)
    return tour

def generate_random_tour():
    tour = list(range(len(CITIES)))
    random.shuffle(tour)
    tour.append(tour[0]) 
    return tour

def tsp_cost(tour):
    cost = 0
    for i in range(len(tour) - 1):
        cost += DIST_MATRIX[tour[i], tour[i + 1]]
    return cost

best_tour = None
best_cost = float('inf')

for _ in range(10):
    start_city = random.choice(range(len(CITIES)))
    
    greedy_tour = greedy_tsp(start_city)
    greedy_cost = tsp_cost(greedy_tour)
    
    logging.info(f"Tour starting from city {start_city}: length {greedy_cost:.2f} km. Found a path of {len(greedy_tour) - 1} steps")
    
    if greedy_cost < best_cost:
        best_cost = greedy_cost
        best_tour = greedy_tour

logging.info(f"The best tour found has a length of {best_cost:.2f} km. Found a path of {len(greedy_tour) - 1} steps")


INFO:root:Tour starting from city 75: length 43142.53 km. Found a path of 167 steps
INFO:root:Tour starting from city 99: length 43754.55 km. Found a path of 167 steps
INFO:root:Tour starting from city 73: length 45479.78 km. Found a path of 167 steps
INFO:root:Tour starting from city 46: length 44662.17 km. Found a path of 167 steps
INFO:root:Tour starting from city 95: length 44700.83 km. Found a path of 167 steps
INFO:root:Tour starting from city 80: length 43689.57 km. Found a path of 167 steps
INFO:root:Tour starting from city 106: length 42548.23 km. Found a path of 167 steps
INFO:root:Tour starting from city 21: length 42137.49 km. Found a path of 167 steps
INFO:root:Tour starting from city 91: length 43410.46 km. Found a path of 167 steps
INFO:root:Tour starting from city 155: length 45194.82 km. Found a path of 167 steps
INFO:root:The best tour found has a length of 42137.49 km. Found a path of 167 steps


## Optimized Inver-Over - Slow but more accurate solution

This code uses an initial local improvement (had to be toned down a bit because of very long execution times) and then applies inver-over.  

Please note that the parameters may not always be optimal and might need changes depending on the problem.  

Population is half random and half computed with a greedy euristic.

In [ ]:
def inver_over_tsp(CITIES, DIST_MATRIX, population_size, generations, 
                        p_mutation, max_no_improve, progress_interval):
    
    #Faster version of inver over
    num_cities = len(CITIES)
    
    def create_random_tour():
        #Random path generation
        tour = list(range(num_cities))
        random.shuffle(tour)
        return tour
    
    def create_smart_individual():
        #We use a greedy approach for a better starting point
        unvisited = set(range(num_cities))
        tour = []
        current = random.randrange(num_cities)
        tour.append(current)
        unvisited.remove(current)
        
        while unvisited:
            # Take only the 2 closest cities to be faster
            candidates = sorted(unvisited, 
                             key=lambda x: DIST_MATRIX[current, x])[:2]
            current = random.choice(candidates)
            tour.append(current)
            unvisited.remove(current)
            
        return tour
    
    def fast_local_improvement(tour):
        for _ in range(10):  # limiting the number of iterations
            # random choice of the segment
            positions = random.sample(range(1, len(tour) - 2), min(30, len(tour) - 3))
            
            for i in positions:
                for j in range(i + 1, min(i + 15, len(tour) - 1)):
                    change = -DIST_MATRIX[tour[i-1], tour[i]] \
                            -DIST_MATRIX[tour[j], tour[j+1]] \
                            +DIST_MATRIX[tour[i-1], tour[j]] \
                            +DIST_MATRIX[tour[i], tour[j+1]]
                    
                    if change < -0.05:
                        tour[i:j+1] = reversed(tour[i:j+1])
                        break
                        
        return tour
    
    def smart_mutation(tour):
        #Mutation that changes nearby segments
        if len(tour) <= 3:
            return tour
        
        # choose random point and swap the segment
        pos = random.randint(1, len(tour) - 3)
        length = random.randint(2, min(5, len(tour) - pos - 1))
        tour[pos:pos+length] = reversed(tour[pos:pos+length])
        return tour
    
    def inver_over_operation(parent, population):
        #Inver-over procedure
        offspring = parent.copy()
        
        if random.random() < p_mutation:
            return smart_mutation(offspring)
            
        city_pos = random.randint(0, num_cities - 1)
        city = offspring[city_pos]
        
        # pick random parent from population
        other_parent = random.choice(population)
        other_pos = other_parent.index(city)
        next_pos = (other_pos + 1) % len(other_parent)
        next_city = other_parent[next_pos]
        
        next_city_pos = offspring.index(next_city)
        
        if next_city_pos != city_pos:
            # inversion
            if next_city_pos > city_pos:
                offspring[city_pos:next_city_pos+1] = reversed(offspring[city_pos:next_city_pos+1])
            else:
                offspring[next_city_pos:city_pos+1] = reversed(offspring[next_city_pos:city_pos+1])
        
        return offspring
    
    def print_progress(gen, best_length, best_tour, elapsed_time):
        #Prints current path
        print(f"\nGeneration {gen}:")
        print(f"Best length: {best_length:.2f} km")

    
    print("Initializing population...")
    initial_pop_size = population_size * 2
    #half population random and half with greedy euristic for better varaibility
    population = ([create_smart_individual() for _ in range(initial_pop_size//2)] +
                 [create_random_tour() for _ in range(initial_pop_size//2)])
    
    print("Applying initial local improvement...")
    population = [fast_local_improvement(tour) for tour in tqdm(population)]
    
    population = sorted(population, key=tsp_cost)[:population_size]
    
    best_tour = population[0]
    best_length = tsp_cost(best_tour)
    generations_without_improvement = 0
    steps = 0 
    start_time = time()
    
    print("\nStarting evolution...")
    print(f"Initial length: {best_length:.2f} km")
    
    pbar = tqdm(range(generations), desc="Generations")
    
    for gen in pbar:
        if generations_without_improvement >= max_no_improve:
            print(f"\nEarly stopping after {gen} generations without improvements")
            break
            
        new_population = []
        improved = False
        
        for parent in population:
            offspring = inver_over_operation(parent, population)
            steps += 1 
            
            if random.random() < 0.1:  # 10% probability of applying local improvement
                offspring = fast_local_improvement(offspring)
                
            new_population.append(offspring)
            
            offspring_length = tsp_cost(offspring)
            if offspring_length < best_length:
                best_length = offspring_length
                best_tour = offspring.copy()
                improved = True
                pbar.set_postfix({'best': f'{best_length:.2f}km'})
        
        if improved:
            generations_without_improvement = 0
        else:
            generations_without_improvement += 1
        
        combined = population + new_population
        population = sorted(combined, key=tsp_cost)[:population_size]
        
        if gen % progress_interval == 0:
            elapsed_time = time() - start_time
            print_progress(gen, best_length, best_tour + [best_tour[0]], elapsed_time)
    
    best_tour.append(best_tour[0])
    #total_time = time() - start_time
    
    return best_tour, best_length, gen, steps 

best_tour, best_length, generations_used, total_steps = inver_over_tsp(
    CITIES, 
    DIST_MATRIX,
    population_size=350, #population size (will be times 2)
    generations=5000, #number of gen
    p_mutation=0.15, #probability of mutation 
    max_no_improve=150, #stops after max no improve
    progress_interval=1000 #prints feedback after n generations, tqdm should handle that anyway
)

print("\nOptimization completed!")
print(f"Total length: {best_length:.2f} km")
print(f"Total number of steps: {total_steps}\n")



Initializing population...
Applying initial local improvement...


100%|██████████| 700/700 [00:01<00:00, 356.35it/s]



Starting evolution...
Initial length: 39313.86 km


Generations:   0%|          | 1/5000 [00:00<14:18,  5.82it/s, best=38190.28km]


Generation 0:
Best length: 38254.63 km


Generations:   7%|▋         | 366/5000 [01:09<14:33,  5.30it/s, best=34950.13km]


Early stopping after 366 generations without improvements

Optimization completed!
Total length: 34950.13 km
Total number of steps: 128100

